In [ ]:
from IPython.display import Image

このチュートリアルでは、「畳み込みニューラルネットワーク (Convolutional Neural Network:CNN)を用いた画像認識タスク」をテーマに取り上げ、CNTK でどのように実装するのかについて、step by step で学んでいただくことを目的にしています。

1. ディープラーニングのワークフローを順を追って見ていき、CNTK の Python API でそれぞれどう実装していくのかを見ていきます。
    1. reader (入力データの定義)
    2. network (モデルを定義)
    3. trainer (モデルをどう学習するかを定義)

2. 一通りワークフローの流れを理解したら、次に、各種パラメータを変更したり、学習テクニックなどを使ってみたり、あるいは、新しくネットワークを
自分自身で設計するなどしてより高い認識精度が出せるようにチャレンジしていただきます。



# 使用するデータセット

本ハンズオンでは、画像認識タスクのベンチマークとしてよく使われる「CIFAR-10」という画像データセットを使用します。

In [ ]:
# Figure 1
Image(url="https://cntk.ai/jup/201/cifar-10.png", width=500, height=500)

CIFAR-10は、約 8000 万枚の画像がある 80 Million Tiny Images 
からサブセットとして約 6 万枚の画像を抽出してラベル付けしたものです。
CIFAR-10 は、上図のように、10 クラスがラベル付けされており、訓練画像: 50000枚
(各クラス 5000枚)、テスト画像: 10000枚(各クラス 1000枚) に分割されています。

In [ ]:
Image(url="https://cntk.ai/jup/201/CNN.png")

In [ ]:
from __future__ import print_function
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import PIL
import sys
#from cntk.utils import *


# 1. データ読み取り (reader)
さて、それではこれからディープラーニングのワークフローを順を追って見ていきましょう。
まず、モデルが学習するためには、一般的に下記に示すような、各画像とその画像の正解ラベルが対応しているようなフォーマットを
入力データ (ここで map text fileと呼ぶ)として想定しています。
※ 生のデータセットからモデルが学習しやすい以下のようなフォーマットに事前に変換すること前処理と呼びます。


<画像データのフルパス> <タブ> <正解ラベル>

Example of a map text file:

    S:\data\CIFAR-10\train\00001.png	9
    S:\data\CIFAR-10\train\00002.png	9
    S:\data\CIFAR-10\train\00003.png	4
    S:\data\CIFAR-10\train\00004.png	1
    S:\data\CIFAR-10\train\00005.png	1

また、CNN で画像を学習させる際、学習画像をずらしたりぼかしたり、色々と変形を加えるなどして学習画像を増やす (このことを data augmentationと呼びます) ことで、認識をロバスト(頑健性を高める) にするというテクニックが一般的によく使われています。

Data Augmentation の例：
* アスペクト比の変更
* 回転 (ロール・ピッチ)
* ぼかし
* ノイズ付与
* ずらし ・・・etc.

CNTK 上で画像データを読み取る際は、1. map text file を取ってきて、2. 画像データに対して data augmentation をかける、といった処理
を平行して行えるような reader を作ります。


In [ ]:
from cntk.io import MinibatchSource, ImageDeserializer, StreamDef, StreamDefs
import cntk.io.transforms as xforms 



# model dimensions
image_height = 32
image_width  = 32
num_channels = 3
num_classes  = 10


#
# Define the reader for both training and evaluation action.
#
def create_reader(map_file, mean_file, train):
    print("Reading map file:", map_file)
    print("Reading mean file:", mean_file)

    # transformation pipeline for the features has jitter/crop only when training
    transforms = []
    if train:
        transforms += [
            xforms.crop(crop_type='randomside', side_ratio=0.8, jitter_type='uniRatio') # train uses data augmentation (translation only)
        ]
    transforms += [
        xforms.scale(width=image_width, height=image_height, channels=num_channels, interpolations='linear'),
        xforms.mean(mean_file)
    ]
    # deserializer
    return MinibatchSource(ImageDeserializer(map_file, StreamDefs(
        features = StreamDef(field='image', transforms=transforms), # first column in map file is referred to as 'image'
        labels   = StreamDef(field='label', shape=num_classes)      # and second as 'label'
    )))


# 2. モデルの定義 (network)
続いて、モデルの定義をするパートです。
上図で示している CNN をストレートに実装したものが以下になります。

In [ ]:
from cntk.layers import Convolution, MaxPooling, Dense
from cntk.initializer import glorot_uniform
from cntk.ops import relu

def create_model(input, out_dims):
    
    net = Convolution((5,5), 32, init=glorot_uniform(), activation=relu, pad=True)(input)
    net = MaxPooling((3,3), strides=(2,2))(net)

    net = Convolution((5,5), 32, init=glorot_uniform(), activation=relu, pad=True)(net)
    net = MaxPooling((3,3), strides=(2,2))(net)

    net = Convolution((5,5), 64, init=glorot_uniform(), activation=relu, pad=True)(net)
    net = MaxPooling((3,3), strides=(2,2))(net)
    
    net = Dense(64, init=glorot_uniform())(net)
    net = Dense(out_dims, init=glorot_uniform(), activation=None)(net)
    
    return net

## Layers ライブラリ を使ってモデルを定義しよう

CNTK には、簡潔にネットワークを定義するために、一般的な (例えば、全結合層や convolution 層など) ニューラルネットワークにおける各層が "Layers" と呼ばれるライブラリの形で事前に定義されています。Layers は関数オブジェクトとして定義されているため、とても巨大で複雑な**関数の集合体**であるニューラルネットワークをより直感的に書くことができます。


## Sequential()
Sequential() は、ネットワークをコンパクトに書く上で非常に便利なオペレーションです。


例えば、隠れ層のユニット数 2048、出力層のユニット 9000で、活性化関数がすべてsigmoid 関数である  
4層順伝播型ニューラルネットワークを実装しようとした場合、以下のように書くことができます。


```py
my_model = Sequential ([
# 活性化関数に sigmoid を使う 4 つの隠れ層
    Dense(2048, activation=sigmoid),  
    Dense(2048, activation=sigmoid),
    Dense(2048, activation=sigmoid),
    Dense(2048, activation=sigmoid),
    # 最後の層(出力層) は活性化関数が softmax
    Dense(9000, activation=softmax)   
])
```

## For()

For() は、定義した lambda 式 をリピート実行することよって、系列モデルを構築することができます。

For() を使うと先ほどの 4層順伝播型ニューラルネットワークもよりコンパクトに書けます。

```py

with default_options(activation=sigmoid):
    my_model = Sequential([
        For(range(4), lambda: Dense(2048)),
        Dense(9000, activation=softmax)
    ])
      
```


## Sequential() や For() を使って、CNN モデルを書いてみよう！
今紹介した便利な関数 Sequential() や For() を使って上記のCNN (create_model() )を以下に書き換えてみましょう！

以下のPython API Reference に各モジュールの使い方がより詳細に記載されているので躓いたら参考にしてみてください。

[参考情報] Python API Reference 
https://www.cntk.ai/pythondocs/cntk.html#

In [ ]:

def create_basic_model(input, out_dims):

    
    `````````````````
    ここに書いてみよう
    `````````````````

    return model(input)

# 3. trainer (and evaluator)

## Trainer を使って、モデルを学習しよう

Trainer は学習に必要な全てのものをカプセル化するクラスです。それは主に以下のような
ものを保持します。
    
* モデル (network パートで定義したモデル)
* 指標: 損失関数 (loss function) と 分類誤差 (clasification error)
* 学習のハイパーパラメータ
    * 学習率
    * 学習アルゴリズム
    * 正則化項
    

    
    
**[復習]** ニューラルネットワークの学習
1. 訓練データの中からランダムに一部のデータ(ミニバッチ)を選び出しモデルに渡す (MinibachSource クラスの next_minibatch())
2. ミニバッチを入力層→出力層へと順番に伝播させていき、最後の出力層で値を出力する
3. 出力結果と正解ラベルを比較し、損失関数を減らすために、各重みパラメータの勾配を計算する
4. 重みパラメータを勾配方向に微小量だけ更新する (trainer クラスの train_minibatch())
5. 1-4 を繰り返す


In [ ]:
#
# Train and evaluate the network.
#

from cntk import momentum_sgd, learning_rate_schedule, UnitType, momentum_as_time_constant_schedule
from cntk import Trainer
from cntk import cross_entropy_with_softmax, classification_error, input_variable, softmax, element_times
from cntk.logging import ProgressPrinter, log_number_of_parameters


def train_and_evaluate(reader_train, reader_test, max_epochs, model_func):
    
    # Input variables denoting the features and label data
    input_var = input_variable((num_channels, image_height, image_width))
    label_var = input_variable((num_classes))

    # Normalize the input
    feature_scale = 1.0 / 256.0
    input_var_norm = element_times(feature_scale, input_var)
    
    # apply model to input
    z = model_func(input_var_norm, out_dims=10)

    #
    # Training action
    #

    # loss and metric
    ce = cross_entropy_with_softmax(z, label_var)
    pe = classification_error(z, label_var)

    # training config
    epoch_size     = 50000
    minibatch_size = 64

    # Set training parameters

    lr_per_minibatch       = learning_rate_schedule([0.01]*10 + [0.003]*10 + [0.001], UnitType.minibatch, epoch_size)
    momentum_time_constant = momentum_as_time_constant_schedule(-minibatch_size/np.log(0.9))
    l2_reg_weight          = 0.001
    
    # trainer object
    learner = momentum_sgd(z.parameters, 
                           lr = lr_per_minibatch, momentum = momentum_time_constant, 
                           l2_regularization_weight=l2_reg_weight)
    progress_printer = ProgressPrinter(tag='Training', num_epochs=max_epochs)
    trainer = Trainer(z, (ce, pe), [learner], [progress_printer])

    # define mapping from reader streams to network inputs
    input_map = {
        input_var: reader_train.streams.features,
        label_var: reader_train.streams.labels
    }

    log_number_of_parameters(z) ; print()

    # Get minibatches of image to train with and perform model training
    batch_index = 0
    plot_data = {'batchindex':[], 'loss':[], 'error':[]}
    for epoch in range(max_epochs):       # loop over epochs
        sample_count = 0
        while sample_count < epoch_size:  # loop over minibatches in the epoch
            data = reader_train.next_minibatch(min(minibatch_size, epoch_size - sample_count), input_map=input_map) # fetch minibatch.
            trainer.train_minibatch(data)                                   # update model with it
            sample_count += data[label_var].num_samples                     # count samples processed so far
            
            # For visualization...            
            plot_data['batchindex'].append(batch_index)
            plot_data['loss'].append(trainer.previous_minibatch_loss_average)
            plot_data['error'].append(trainer.previous_minibatch_evaluation_average)
            
            batch_index += 1
        trainer.summarize_training_progress()
        
    #
    # Evaluation action
    #
    epoch_size     = 10000
    minibatch_size = 16

    # process minibatches and evaluate the model
    metric_numer    = 0
    metric_denom    = 0
    sample_count    = 0
    minibatch_index = 0

    while sample_count < epoch_size:
        current_minibatch = min(minibatch_size, epoch_size - sample_count)

        # Fetch next test min batch.
        data = reader_test.next_minibatch(current_minibatch, input_map=input_map)

        # minibatch data to be trained with
        metric_numer += trainer.test_minibatch(data) * current_minibatch
        metric_denom += current_minibatch

        # Keep track of the number of samples processed so far.
        sample_count += data[label_var].num_samples
        minibatch_index += 1

    print("")
    print("Final Results: Minibatch[1-{}]: errs = {:0.1f}% * {}".format(minibatch_index+1, (metric_numer*100.0)/metric_denom, metric_denom))
    print("")
    
    # Visualize training result:
    window_width            = 32
    loss_cumsum             = np.cumsum(np.insert(plot_data['loss'], 0, 0)) 
    error_cumsum            = np.cumsum(np.insert(plot_data['error'], 0, 0)) 

    # Moving average.
    plot_data['batchindex'] = np.insert(plot_data['batchindex'], 0, 0)[window_width:]
    plot_data['avg_loss']   = (loss_cumsum[window_width:] - loss_cumsum[:-window_width]) / window_width
    plot_data['avg_error']  = (error_cumsum[window_width:] - error_cumsum[:-window_width]) / window_width
    
    plt.figure(1)
    plt.subplot(211)
    plt.plot(plot_data["batchindex"], plot_data["avg_loss"], 'b--')
    plt.xlabel('Minibatch number')
    plt.ylabel('Loss')
    plt.title('Minibatch run vs. Training loss ')

    plt.show()

    plt.subplot(212)
    plt.plot(plot_data["batchindex"], plot_data["avg_error"], 'r--')
    plt.xlabel('Minibatch number')
    plt.ylabel('Label Prediction Error')
    plt.title('Minibatch run vs. Label Prediction Error ')
    plt.show()
    
    return softmax(z)

## 学習/テストを開始する
上で作った train_and_evaluate() に対し、入力データ(CIFAR-10)を create_reader 経由で渡して、
学習およびテストしてみましょう。


In [ ]:
data_path = os.path.join('data', 'CIFAR-10')
reader_train = create_reader(os.path.join(data_path, 'train_map.txt'), os.path.join(data_path, 'CIFAR-10_mean.xml'), True)
reader_test  = create_reader(os.path.join(data_path, 'test_map.txt'), os.path.join(data_path, 'CIFAR-10_mean.xml'), False)

pred = train_and_evaluate(reader_train, reader_test, max_epochs=10, model_func=create_model)

## 学習済みモデルを使った予測
先ほど学習したモデルを使って実際にいつくつかのテスト画像を分類させてみましょう。

In [ ]:
# Figure 6
Image(url="https://cntk.ai/jup/201/00014.png", width=64, height=64)

In [ ]:
def eval(pred_op, image_path):
    label_lookup = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    image_mean   = 133.0
    image_data   = np.array(PIL.Image.open(image_path), dtype=np.float32)
    image_data  -= image_mean
    image_data   = np.ascontiguousarray(np.transpose(image_data, (2, 0, 1)))
    
    result       = np.squeeze(pred_op.eval({pred_op.arguments[0]:[image_data]}))
    
    # Return top 3 results:
    top_count = 3
    result_indices = (-np.array(result)).argsort()[:top_count]
    
    
    print("Top 3 predictions:")
    for i in range(top_count):
        print("\tLabel: {:10s}, confidence: {:.2f}%".format(label_lookup[result_indices[i]], result[result_indices[i]] * 100))

In [ ]:
eval(pred, "data/CIFAR-10/test/00014.png")

## Dropout
Dropout は、過学習を抑制するために提唱された手法です。
過学習 (Overfitting) とは、機械学習において、訓練データに対して学習されるが、未知のデータに対して適合できていない (汎化できていない) 状態を指します。特にニューラルネットワークのようにパラメータを大量に持ち、自由度の高いモデルは **過学習に陥りやすい** と言われています。

Dropout では、ニューラルネットワークを学習する際に、ノードをランダムに消去しながら学習する手法です。ある更新で隠れ層のノードのうちいくつかを消去して学習を行い、次の更新では別のノードを消去して学習を行うことを繰り返します。これにより学習時におけるニューラルネットワークの自由度を小さくして汎化性能を上げ、過学習を抑制することができます。

◇ Tips

隠れ層においては、一般的に 50% 程度のノードを消去すると良いと言われています。また、当初 Dropout は全結合層のみに適用されていましたが、最近の研究では、畳み込み層などにも適用しても同様に性能を向上させることが確認されています。

In [ ]:
def create_basic_model_with_dropout(input, out_dims):

    with default_options(activation=relu):
        model = Sequential([
            For(range(3), lambda i: [
                Convolution((5,5), [32,32,64][i], init=glorot_uniform(), pad=True),
                Dropout(0.5),
                MaxPooling((3,3), strides=(2,2))
            ]),
            Dense(64, init=glorot_uniform()),
            Dropout(0.5),
            Dense(out_dims, init=glorot_uniform(), activation=None)
        ])

    return model(input)

## VGG
VGG は、畳み込み層とプーリング層から構成されるシンプルな CNN です。ただし、重みのある層 (畳み込み層や全結合層)を全部で 16 層 (もしくは 19　層)まで重ねてディープ　にしています (層の深さに応じて、「VGG16」や「VGG19」と呼ばれます)。
VGG では、フィルターサイズ 3x3 の畳み込み層　+ Max Pooling を数回繰り返し後、全結合層を経由して結果を出力します。
VGG は性能が高いだけでなく、このようにとてもシンプルな構成で応用性が高いため、多くの技術者は VGG ベースのネットワークを好んで使っています。


| VGG9          |
| ------------- |
| conv3-64      |
| conv3-64      |
| max3          |
|               |
| conv3-96      |
| conv3-96      |
| max3          |
|               |
| conv3-128     |
| conv3-128     |
| max3          |
|               |
| FC-1024       |
| FC-1024       |
|               |
| FC-10         |


上図は、重みのある層が全部で9層ある VGG ベースのモデル図になります。
こちらを参照しながら VGG モデルを作ってみましょう。また、作ったモデルで学習・テストをし、認識精度の違いを比べてみましょう！



In [ ]:
def create_vgg9_model(input, out_dims):
    
    `````````````````
    ここに書いてみよう
    `````````````````
    return model(input)